In [ ]:
import os
os.chdir('../../..')

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
from examples.three_tank_system.data_gen import ThreeTankDataGenerator
import pandas as pd
import examples.three_tank_system.constants as const
import pysindy as ps
import numpy as np
import plotly.express as px
from ipywidgets import interact
import ipywidgets as widgets
from plotly.subplots import make_subplots
import plotly.graph_objects as go
%load_ext autoreload
%autoreload 2

In [ ]:
df = pd.read_parquet(const.X_SPACE_DATA_PATH)

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
def _create_ts_plot(idx):
    df_plot = df[df[const.UID_INITIAL_STATE_COL_NAME] == idx]
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
    # signal 1
    for col, name in zip(const.Z_COL_NAMES, ['h1(t)', 'h2(t)', 'h3(t)']):
        fig.add_trace(go.Scatter(x=df_plot.time, y=df_plot[col], name=name,
                      mode="lines", opacity=1),
            row=1, col=1)

    for col, name in zip(const.Z_DOT_COL_NAMES, ['h1_dot(t)', 'h2_dot(t)', 'h3_dot(t)']):
        fig.add_trace(go.Scatter(x=df_plot.time, y=df_plot[col], name=name,
                      mode="lines", opacity=1),
            row=2, col=1)

    fig.update_xaxes(title_text='time')
    fig.update_yaxes(title_text='x', row=1)
    fig.update_yaxes(title_text='x_dot', row=2)
    fig.update_layout(title_text="Latent neuron activations vs. hidden states", showlegend=True)
    fig.show()

interact(_create_ts_plot, idx=list(range(const.NUMBER_INITIAL_STATES)))

# Lets try the pysindy exercise in the z-space!

In [ ]:
library_functions = [
#     lambda x : np.exp(x),
    lambda x : 1./x,
    lambda x : x,
    lambda x : np.sin(x),
    lambda x : np.cos(x),
    lambda x,y : np.sin(x+y),
    lambda x,y : np.cos(x+y),
#     lambda x,y : np.sign(x-y)*np.sqrt(np.abs(x-y)),
    lambda x: x**2,
#     lambda x: np.sqrt(x),
    lambda x,y: x*y,
    lambda x,y: x**2*y,
    lambda x,y: x*y**2
]
library_function_names = [
#     lambda x : 'exp(' + x + ')',
    lambda x : '1/' + x,
    lambda x : x,
    lambda x : f'sin({x})',
    lambda x : f'cos({x})',
    lambda x,y : 'sin(' + x + '+' + y + ')',
    lambda x,y : f'cos({x}+{y})',
#     lambda x,y : 'sign('+x+'-'+y+')*sqrt('+x+' - '+y+')',
    lambda x: x+'^2',
#     lambda x: f'sqrt({x})',
    lambda x,y: f'{x}*{y}',
    lambda x,y: f'{x}^2*{y}',
    lambda x,y: f'{x}*{y}^2'
]
feature_library = ps.CustomLibrary(
    library_functions=library_functions, function_names=library_function_names
)

In [ ]:
optimizer = ps.STLSQ(threshold=.01)
model = ps.SINDy(
    feature_library=feature_library,
    optimizer=optimizer,
    feature_names=["h1", "h2", "h3"],
)
model.fit(df_z[const.Z_COL_NAMES].values, x_dot=df_z[const.Z_DOT_COL_NAMES].values, t=0.01)
model.print()

## NICE!

In [ ]:
feature_library.get_feature_names()

In [ ]:
len(const.X_COL_NAMES)

In [ ]:
df_x = pd.read_parquet(const.X_SPACE_DATA_PATH)

In [ ]:
df_x.info()

In [ ]:
def _create_ts_plot(idx):
    df_plot = df[df[const.UID_INITIAL_STATE_COL_NAME] == idx]
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
    # signal 1
    for col, name in zip(const.X_COL_NAMES, const.X_COL_NAMES):
        fig.add_trace(go.Scatter(x=df_plot.time, y=df_plot[col], name=name,
                      mode="lines", opacity=1), row=1, col=1)
    for col, name in zip(const.XDOT_COL_NAMES, const.XDOT_COL_NAMES):
        fig.add_trace(go.Scatter(x=df_plot.time, y=df_plot[col], name=name,
                      mode="lines", opacity=1), row=2, col=1)
    fig.show()
interact(_create_ts_plot, idx=list(range(const.NUMBER_INITIAL_STATES)))

In [ ]:
df.head()

# Now in the x space!

In [ ]:
df_x = pd.read_parquet(const.X_SPACE_DATA_PATH)

def plot_pics(sample_index):
    df_plot_x = df_x[df_x[const.UID_INITIAL_STATE_COL_NAME] == sample_index].copy().reset_index(drop=True)
    sub_indexes = [i for i in range(5)]
    img_sequence = [df_plot_x[const.X_COL_NAMES].values[i,:].reshape(
        (const.PICTURE_SIZE, const.PICTURE_SIZE)) for i in sub_indexes]
    fig = px.imshow(np.array(img_sequence),
                    facet_col=0,
#                     binary_string=True,
                    labels={'facet_col':'sigma'}
                   )
    for i, index in enumerate(sub_indexes):
        fig.layout.annotations[i]['text'] = f'Timestamp {index}'
    fig.show()

interact(plot_pics, sample_index=range(const.NUMBER_INITIAL_STATES))

In [ ]:
def plot_pics(sample_index):
    df_plot_x = df_x[df_x[const.UID_INITIAL_STATE_COL_NAME] == sample_index].copy().reset_index(drop=True)
    sub_indexes = [i for i in range(20,30)]
    img_sequence = [df_plot_x[const.XDOT_COL_NAMES].values[i,:].reshape(
        (const.PICTURE_SIZE, const.PICTURE_SIZE)) for i in sub_indexes]
    fig = px.imshow(np.array(img_sequence),
                    facet_col=0,
#                     binary_string=True,
                    labels={'facet_col':'sigma'}
                   )
    for i, index in enumerate(sub_indexes):
        fig.layout.annotations[i]['text'] = f'Timestamp {index}'
    fig.show()

interact(plot_pics, sample_index=range(const.NUMBER_INITIAL_STATES))

In [ ]:
df_x = pd.read_parquet(const.X_SPACE_DATA_PATH)

def plot_pics(sample_index):
    df_plot_x = df_x[df_x[const.UID_INITIAL_STATE_COL_NAME] == sample_index].copy().reset_index(drop=True)
    sub_indexes = [0, 10, 20, 30, 40]
    img_sequence = [df_plot_x[const.X_COL_NAMES].values[i,:].reshape(
        (const.PICTURE_SIZE, const.PICTURE_SIZE)) for i in sub_indexes]
    fig = px.imshow(np.array(img_sequence),
                    facet_col=0,
#                     binary_string=True,
                    labels={'facet_col':'sigma'}
                   )
    for i, index in enumerate(sub_indexes):
        fig.layout.annotations[i]['text'] = f'Timestamp {index}'
    fig.show()

interact(plot_pics, sample_index=range(const.NUMBER_INITIAL_STATES))

## What about the derivatives in x?

In [ ]:
def plot_pics(sample_index):
    df_plot_x = df_x[df_x[const.UID_INITIAL_STATE_COL_NAME] == sample_index].copy().reset_index(drop=True)
    sub_indexes = [0, 25, 50, 75, 100]
    img_sequence = [df_plot_x[const.XDOT_COL_NAMES].values[i,:].reshape(
        (const.PICTURE_SIZE, const.PICTURE_SIZE)) for i in sub_indexes]
    fig = px.imshow(np.array(img_sequence),
                    facet_col=0,
#                     binary_string=True,
                    labels={'facet_col':'sigma'}
                   )
    for i, index in enumerate(sub_indexes):
        fig.layout.annotations[i]['text'] = f'Timestamp {index}'
    fig.show()

interact(plot_pics, sample_index=range(const.NUMBER_INITIAL_STATES))

In [ ]:
x_dot = df_x[const.XDOT_COL_NAMES].values
x = df_x[const.X_COL_NAMES].values

In [ ]:
i = 10

In [ ]:
px.imshow(x[i,:].reshape(100, 100))

In [ ]:
px.imshow(x_dot[i,:].reshape(100, 100))

In [ ]:
df_x[df_x[const.UID_INITIAL_STATE_COL_NAME] == 0]

In [ ]:
df_x.info()

In [ ]:
pd.Series([df_x[col].max() for col in const.X_COL_NAMES]).value_counts()

In [ ]:
df_test = df_x[df_x[const.UID_INITIAL_STATE_COL_NAME] == 0]

In [ ]:
df_test[const.X_COL_NAMES].std()[df_test[const.X_COL_NAMES].std()>0.069]

In [ ]:
x = np.array([[1, 2, 3],[4,5,6],[7,8,9]])
x

In [ ]:
x.ravel()

In [ ]:
33*100+1

In [ ]:
set([df_x[df_x[const.UID_INITIAL_STATE_COL_NAME] == 0][f'x_{i}'].max() for i in range(1000)])

In [ ]:
def plot_stuff(i=3504):
    fig = px.scatter(df_x[df_x[const.UID_INITIAL_STATE_COL_NAME] == 0][f'x_{i}'])
#     fig = px.scatter(df_x[f'x_{i}'])

    fig.show()
interact(plot_stuff, i = [i for i in range(10000)])
    

In [ ]:
def plot_stuff(i=3504):
    fig = px.scatter(df_x[df_x[const.UID_INITIAL_STATE_COL_NAME] == 0][f'xdot_{i}'])
#     fig = px.scatter(df_x[f'x_{i}'])

    fig.show()
interact(plot_stuff, i = [i for i in range(10000)])